In [33]:
import torch
import torch.nn as nn

In [34]:
# Batch size x Height x Width 크기의 텐서 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('Tensor size: {}'.format(inputs.size()))

Tensor size: torch.Size([1, 1, 28, 28])


In [35]:
conv1 = nn.Conv2d(1, 32, 3, padding = 1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [36]:
conv2 = nn.Conv2d(32, 64, kernel_size= 3, padding = 1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [37]:
pool = nn.MaxPool2d(2, 2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [38]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [39]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [40]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [41]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [42]:
print(out.size(0))
print(out.size(1))
print(out.size(2))
print(out.size(3))

1
64
7
7


In [43]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1) # => 1, 64*7*7
print(out.shape)

torch.Size([1, 3136])


In [44]:
fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [45]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

torch.manual_seed(123)

if device == 'mps':
    print('Yes mps')
    torch.cuda.manual_seed_all(123)

Yes mps


In [46]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root = './data', train = True, transform = transforms.ToTensor(), download = True )
mnist_test = dsets.MNIST(root = './data', train = False, transform = transforms.ToTensor(), download = True)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [47]:
data_loader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

In [48]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, padding = 1, stride = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, padding = 1, stride = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        )

        # Fully Connection 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)

        # Fully connection 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # Fully connection을 위해 Flatten
        out = self.fc(out)
        return out

In [49]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

총 배치의 수: 600


In [50]:
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch # 현재 배치의 비용을 전체 배치 수로 나누어 누적
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.21943076
[Epoch:    2] cost = 0.0645758212
[Epoch:    3] cost = 0.0470495895
[Epoch:    4] cost = 0.0386406295
[Epoch:    5] cost = 0.031620983
[Epoch:    6] cost = 0.026266763
[Epoch:    7] cost = 0.0233989675
[Epoch:    8] cost = 0.0187653694
[Epoch:    9] cost = 0.0159850102
[Epoch:   10] cost = 0.013141457
[Epoch:   11] cost = 0.0112494892
[Epoch:   12] cost = 0.0105829509
[Epoch:   13] cost = 0.00775105087
[Epoch:   14] cost = 0.00897533726
[Epoch:   15] cost = 0.00666138064


In [52]:
with torch.no_grad():
    # 테스트 데이터를 모델에 입력하기 위한 준비
    # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동
    Y_test = mnist_test.targets.to(device)
    print(X_test.shape)
    print(Y_test.shape)

    prediction = model(X_test)
    correct_pred = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_pred.float().mean()
    print('Accuracy: ', accuracy.item())

torch.Size([10000, 1, 28, 28])
torch.Size([10000])
Accuracy:  0.9868999719619751
